In [ ]:
import requests
import pandas as pd
import sys
import urllib
import pickle
import time

## Gather

image_predictions.tsv is hosted on Udacity's servers  
Should be downloaded programmatically using the Requests library  
URL: 
https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv
<br>
<br>

In [ ]:
url ='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
url_list = url.split('/')
image_predictions_file_name = url_list[-1]

response = requests.get(url)

with open(image_predictions_file_name, mode='wb') as file:
    file.write(response.content)

In [ ]:
# production 2075 rows, obserations
image_predictions_df = pd.read_csv('image-predictions.tsv', sep='\t')

image_predictions_df_clean = image_predictions_df.copy()
image_predictions_df_clean.info()

## Assess

* tweet_id pandas series, column type - int64 not object

## Clean

### Define

* change tweet_id pandas series, column from int64 to object

### Code

In [ ]:
image_predictions_df_clean.tweet_id = \
image_predictions_df_clean.tweet_id.apply(str)

### Test

In [ ]:
image_predictions_df_clean.info()

Success
* tweet_id pandas series, column no longer int64 changed to object

## Assess

Requirement compliance  

Data Analyst Nanodegree Program  
8\. Data Wrangling  
Project  
2\. Project Motivation  
Key Points  
You only want original ratings ... that have images.

some jpg_url(s) in image-predictions.tsv are unreachable 

## Clean

### Define

* identify and capture unreachable image_predictions_df_clean jpg_url(s)

### Code

In [ ]:
# supports echoes during hour plus Jupyter cell run time
break_count = 0

# capture the invalid jpg_url(s) here
bad_url_dictionary = {}

# eventually we will clean, delete the rows, observations
# with the invalid jpg_url(s).
# first level set with the before DataFrame metrics
print("image_predictions_df_clean.shape")
print(image_predictions_df_clean.shape)
print()

for index, row in image_predictions_df_clean.iterrows():
    # supports spurious network exceptions management
    attempts = 0 
    
    bad_url_list = []
    break_count +=1
    # print('row.tweet_id - {}'.format(row.tweet_id))
    # print('row.jpg_url - {}'.format(row.jpg_url))

    # while loop to mitigate spurious network issues
    # see nested try except clauses
    while attempts < 5:
        try:
            # troubleshooting development helper, again this takes an hour plus
            if True:
                x = urllib.request.urlopen(row.jpg_url, timeout = 20).getcode()
                # print('x - {}\n'.format(x))
                # x - 200
                # print('type(x) - {}\n'.format(type(x)))
                #        type(x) - <class 'int'>

                # urllib.request.urlopen('https://pbs.twimg.com/media/CWDbv2yU4AARfeH.jpg').getcode()
                # pass
            break
        except urllib.error.HTTPError:
            # capture the invalid jpg_url(s)
            bad_url_list.append(row.tweet_id)        
            bad_url_list.append(row.jpg_url)  
            bad_url_dictionary[index] = bad_url_list

            print('HTTPError - sys.exc_info()')
            print(sys.exc_info())
            
            print('index - {}'.format(index))
            print('row.tweet_id - {}'.format(row.tweet_id))
            print('row.jpg_url - {}'.format(row.jpg_url))
            print()
            break
        except urllib.error.URLError:
            # manage spurious network errors
            attempts +=1
            print('URLError - sys.exc_info()')
            print(sys.exc_info())
            
            print('index - {}'.format(index))
            print('row.tweet_id - {}'.format(row.tweet_id))
            print('row.jpg_url - {}'.format(row.jpg_url))
            print()
            # mitigate spurious network issues
            time.sleep(0.5)
        except OSError:
            # manage spurious network errors
            attempts +=1 
            print('OSError - sys.exc_info()')
            print(sys.exc_info())
            
            print('index - {}'.format(index))
            print('row.tweet_id - {}'.format(row.tweet_id))
            print('row.jpg_url - {}'.format(row.jpg_url))
            print()
            # mitigate spurious network issues
            time.sleep(0.5)
        except:
            # process takes 1 hour plus, facilitates running to completion
            # if something missed - display it  
            attempts +=1
            print('except - sys.exc_info()')
            print(sys.exc_info())
            
            print('index - {}'.format(index))
            print('row.tweet_id - {}'.format(row.tweet_id))
            print('row.jpg_url - {}'.format(row.jpg_url))
            print()
            # mitigate spurious network issues
            time.sleep(0.5)
            
    # mitigate spurious network issues
    time.sleep(0.5)

    # echo progress of hour plus Jupyter cell
    if break_count % 100 == 0:
        print('break_count - {}'.format(break_count))
        # pass
       
    # facilitates development of hour plus Jupyter cell
    # decrease 10000 during development
    if break_count > 10000:
        break

## Test

verify unreachable image_predictions_df_clean jpg_url(s) captured

In [ ]:
print('bad_url_dictionary')
print(bad_url_dictionary)
print(len(bad_url_dictionary))

In [ ]:
# took hour(s) to generate the bad_url_dictionary
# pickle write - save to file
with open('bad_url_dictionary.pkl', 'wb') as filename:
    pickle.dump(bad_url_dictionary, filename, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# make sure we can read it
with open('bad_url_dictionary.pkl', 'rb') as filename:
    bad_url_dictionary = pickle.load(filename)
print('bad_url_dictionary')
print(bad_url_dictionary)

In [ ]:
# verify we captured the unreachable urls
for key, jpg_url in bad_url_dictionary.items():
    print('tweet_id - {}'.format(jpg_url[0]))
    print('\tjpg_url - {}'.format(jpg_url[1]))

    try:
        code = urllib.request.urlopen(jpg_url[1], timeout = 20).getcode()
        print('code - {}\n'.format(code))
    except urllib.error.HTTPError:
        # print(sys.exc_info()[1]) 
        print('\t{}\n'.format(sys.exc_info()[1]))

Success  
definitively identified unreachable image_predictions_df_clean jpg_url(s) in the  
programmatically downloaded image-predictions.tsv file

## Assess

* unreachable image-predictions.tsv jpg_url(s)

## Clean

### Define

* remove rows, observations with unreachable image-predictions.tsv jpg_url(s)

### Code

In [ ]:
print("image_predictions_df_clean.shape")
print(image_predictions_df_clean.shape)
print()
 
for key, jpg_url in bad_url_dictionary.items():
    print('tweet_id - {}'.format(jpg_url[0]))
    print('\tjpg_url - {}'.format(jpg_url[1]))
    image_predictions_df_clean = \
    image_predictions_df_clean[image_predictions_df_clean.tweet_id != jpg_url[0]]

print('image_predictions_df_clean.shape')
print(image_predictions_df_clean.shape)
print()

math works 2075 - 2072 - 3 rows removed

### Test

* confirm unreachable image_predictions_df_clean jpg_url(s) removed

In [ ]:
# random hard coded positive control we should get this one    
print("image_predictions_df_clean.loc[image_predictions_df_clean.tweet_id\
                               .isin(['837110210464448512'])]")
print(image_predictions_df_clean.loc[image_predictions_df_clean.tweet_id\
                               .isin(['837110210464448512'])])
print()

# We should not get any of these 
for key, jpg_url in bad_url_dictionary.items():
    print('tweet_id - {}'.format(jpg_url[0]))
    image_predictions_df_clean.loc[image_predictions_df_clean.tweet_id\
                               .isin([jpg_url[0]])]
    # print('x - {}'.format(x))
    # print('type(x) - {}\n'.format(type(x)))
    
    print(image_predictions_df_clean.loc[image_predictions_df_clean.tweet_id\
                               .isin([jpg_url[0]])])
    print()
    
 

Success
* 3 rows, observations with unreachable jpg_url(s) removed

In [ ]:
# save cleaned - image_predictions_df_clean
with open('image_predictions_df_clean.pkl', 'wb') as f:
    pickle.dump(image_predictions_df_clean, f)
    
# Read cleaned pandas DataFrame
with open('image_predictions_df_clean.pkl', 'rb') as f:
    image_predictions_df_clean = pickle.load(f)
    
# make sure we can read it
print('image_predictions_df_clean.shape')
print(image_predictions_df_clean.shape)
print()
